In [1]:
import os
%load_ext autoreload
%autoreload 2
import pandas as pd
import pyspark
import pyarrow

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
os.environ['SPARK_HOME'] = 'C:/Users/saul2/Spark_DF/spark-3.5.5-bin-hadoop3'
os.environ['HADOOP_HOME'] = 'C:/Users/saul2/Spark_DF/spark-3.5.5-bin-hadoop3'
os.environ['JAVA_HOME'] = 'C:/Program Files/Java/jdk1.8.0_202'

In [5]:
#spark = SparkSession.builder.appName("Practice").\
#    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
#    .config("spark.hadoop.hadoop.native.io", "false").getOrCreate()
spark = SparkSession.builder.appName("Practice").getOrCreate()

In [6]:
csv_file = 'Students_Grading_Dataset.csv'
df = spark.read.option("header", True).csv(csv_file)
df.createOrReplaceTempView("Students")

In [7]:
# Show Column Names
query = 'DESC STUDENTS'
result = spark.sql(query)
#result.show()

In [8]:
# Students Older than 21
query = "SELECT * FROM STUDENTS WHERE AGE > 21"
result = spark.sql(query)
#result.show()

In [9]:
# Students with a total_score higher than their department_avg
query = '''
    WITH DEPT_AVG AS (
        SELECT DEPARTMENT, ROUND(AVG(TOTAL_SCORE),2) AVG_SCORE
        FROM STUDENTS 
        GROUP BY DEPARTMENT
    )
    SELECT STUDENTS.DEPARTMENT, STUDENT_ID, FIRST_NAME, LAST_NAME, TOTAL_SCORE, DEPT_AVG.AVG_SCORE
    FROM STUDENTS, DEPT_AVG
    WHERE STUDENTS.DEPARTMENT = DEPT_AVG.DEPARTMENT
        AND STUDENTS.TOTAL_SCORE > DEPT_AVG.AVG_SCORE
    ORDER BY STUDENTS.DEPARTMENT, STUDENT_ID
    '''
result = spark.sql(query)
result.show()

+----------+----------+----------+---------+-----------+---------+
|DEPARTMENT|STUDENT_ID|FIRST_NAME|LAST_NAME|TOTAL_SCORE|AVG_SCORE|
+----------+----------+----------+---------+-----------+---------+
|  Business|     S1001|     Maria|    Brown|      92.29|     74.0|
|  Business|     S1006|     Ahmed|    Jones|      79.78|     74.0|
|  Business|     S1010|      John|    Smith|      74.52|     74.0|
|  Business|     S1013|      John| Williams|       74.8|     74.0|
|  Business|     S1016|     Maria|    Jones|      78.95|     74.0|
|  Business|     S1022|      Sara|    Davis|      87.98|     74.0|
|  Business|     S1026|      Emma|  Johnson|      89.76|     74.0|
|  Business|     S1030|     Ahmed|    Brown|      74.94|     74.0|
|  Business|     S1038|       Ali| Williams|      76.22|     74.0|
|  Business|     S1040|      John|  Johnson|       90.3|     74.0|
|  Business|     S1044|     Maria|    Smith|      88.66|     74.0|
|  Business|     S1053|     Maria|    Smith|      77.51|     7

In [10]:
#result.write.mode("overwrite").saveAsTable("student_results_table")
#result.write.mode("overwrite").parquet("output/student_results.parquet")
#result.coalesce(1).write.mode("overwrite").parquet("output/student_results.parquet")
#result.write.mode("overwrite").csv("output/student_results_csv", header=True)
#result.write.mode("overwrite").csv("C:/output/student_results_csv", header=True)
#result.write.format("parquet").save("file:///C:/Users/saul2/OneDrive/Desktop/PastProjects/SQL_Query_Practice/Student_Grading_Dataset/sdd")

In [15]:
# Make a directory for my files
output_dir = 'results'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [17]:
# Conver my result to a Pandas Dataframe. PySpark is not working well to conver it into a parquet/csv file.
pandas_df = result.toPandas()
path = os.path.join(output_dir, 'student_results.parquet')
pandas_df.to_parquet(path, engine='pyarrow') 
path = os.path.join(output_dir, 'student_results.csv')
pandas_df.to_csv(path, index=False)

In [19]:
query = '''
    SELECT COUNT(*)
    FROM STUDENTS 
    '''
result = spark.sql(query)
result.show()

+--------+
|count(1)|
+--------+
|    5000|
+--------+

